In [1]:
#This notebook will contain functions to build and update a list of mechs current in Mechwarrior Online.
#Mech list pages can be found at:
#Light: https://wiki.mwomercs.com/index.php?title=Light_Mechs&action
#Medium: https://wiki.mwomercs.com/index.php?title=Medium_Mechs&action
#Heavy:https://wiki.mwomercs.com/index.php?title=Heavy_Mechs&action
#Assault: https://wiki.mwomercs.com/index.php?title=Assault_Mechs&action

#all mechs via gamepedia: https://mwo.gamepedia.com/Category:Playable

In [5]:
import requests
from requests import get
import re
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import lxml.html as lh
import pandas as pd

In [87]:
page_string

'<!DOCTYPE html> <html class="client-nojs" lang="en" dir="ltr"> <head> <meta charset="UTF-8"/> <title>View source for Assault Mechs - MWO Wiki</title> <script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script> <script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Assault_Mechs","wgTitle":"Assault Mechs","wgCurRevisionId":1349,"wgRevisionId":0,"wgArticleId":107,"wgIsArticle":false,"wgIsRedirect":false,"wgAction":"edit","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[],"wgBreakFrames":true,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesSho

In [93]:
#url='https://wiki.mwomercs.com/index.php?title=Light_Mechs&action=edit'
url = "https://wiki.mwomercs.com/index.php?title=Assault_Mechs&action=edit"
#has 4 tables, 1 for each weight class
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
page_string = page.text

page_string = page_string.replace("\n", " ")
#page_string
#define regex for searching page text
mech_obj = re.compile(r'===\s[\w\s-]+[\s()A-Z0-9-]*\s===')
tonnage_obj = re.compile(r'Tonnage[\']*:[\s\d+]+')
chasis_obj = re.compile(r'Var\w\wnts[\']+:[\sa-zA-Z0-9-,]+')

#get matching tonnage, variant list using regex pattern
mech_results = mech_obj.finditer(page_string)
tonnage_results = tonnage_obj.finditer(page_string)
chasis_results = chasis_obj.finditer(page_string)

#for result in tonnage_results:
#    print(result.group())

#clean regex results to get desired text for each mech: name, weight, chasis variants
mech_names = [mech_name.group().replace("===", "").strip() for mech_name in mech_results]
mech_weights = [mech_weight.group().replace("\n", "")[-5:].strip() for mech_weight in tonnage_results]
chasis_variants = [chasis.group().replace("\n","")[12:].replace(",","").split() for chasis in chasis_results]

#convert lists to dict as preprocess for converstion to dataframe
mech_dict = {
    "mechs":mech_names,
    "tonnage":mech_weights,
    "variants":chasis_variants
}

print(len(mech_names), len(mech_weights), len(chasis_variants))

for i in range(len(chasis_variants)):
    print(mech_weights[i], mech_names[i], chasis_variants[i])
#mech_df = pd.DataFrame(mech_dict)
#mech_df

27 27 27
80 Awesome (AWS) ['AWS-8Q', 'AWS-8R', 'AWS-8T', 'AWS-8V', 'AWS-9M']
80 Charger (CGR) ['CGR-1A1', 'CGR-1A5', 'CGR-3K']
80 Hatamoto (HTM) ['HTM-27T', 'HTM-27W', 'HTM-28TR']
80 Victor (VTR) ['VTR-9A1', 'VTR-9B', 'VTR-9K', 'VTR-9S']
80 Zeus (ZEU) ['ZEU-5S', 'ZEU-6S', 'ZEU-6T', 'ZEU-9S', 'ZEU-9S2']
85 Battlemaster (BLR) ['BLR-1D', 'BLR-1G', 'BLR-1S', 'BLR-2C', 'BLR-3M', 'BLR-3S']
85 Stalker (STK) ['STK-3F', 'STK-3FB', 'STK-3H', 'STK-4N', 'STK-5M', 'STK-5S']
90 Cyclops (CP) ['CP-10-Q', 'CP-10-Z', 'CP-11-A', 'CP-11-A-DC', 'CP-11-P']
90 Highlander (HGN) ['HGN-732', 'HGN-732B', 'HGN-733', 'HGN-733C', 'HGN-733P']
90 Mauler (MAL) ['MAL-1P', 'MAL-1R', 'MAL-2P', 'MAL-MX90']
95 Banshee (BNC) ['BNC-3E', 'BNC-3M', 'BNC-3S']
95 Nightstar (NSR) ['NSR-10P', 'NSR-9FC', 'NSR-9J', 'NSR-9P', 'NSR-9S']
100 Annihilator (ANH) ['ANH-1A', 'ANH-1E', 'ANH-1P', 'ANH-1X', 'ANH-2A']
100 Atlas (AS7) ['AS7-D', 'AS7-D-DC', 'AS7-K', 'AS7-RS', 'AS7-S']
100 Fafnir (FNR) ['FNR-5', 'FNR-5B', 'FNR-5E', 'FNR-6R', 'FNR-

In [92]:
text = "| === Dire Wolf (DWF) ===  \'\'\'T"
mech_obj = re.compile(r'===\s[\w\s-]+[\s()A-Z0-9-]*\s===')
mech_test = mech_obj.finditer(text)
mech_names = [mech_name.group() for mech_name in mech_results]
mech_names

[]

In [94]:
def get_mech_df(url=None):
    if not url:
        print("must pass URL")
        return

    url=url
    page = requests.get(url)
    page_string = page.text

    mech_obj = re.compile(r'===\s[\w\s-]+[\s()A-Z0-9-]*\s===')
    tonnage_obj = re.compile(r'Tonnage[\']*:[\s\d+]+')
    chasis_obj = re.compile(r'Var\w\wnts[\']+:[\sa-zA-Z0-9-,]+')


    #get matching tonnage, variant list
    mech_results = mech_obj.finditer(page_string)
    tonnage_results = tonnage_obj.finditer(page_string)
    chasis_results = chasis_obj.finditer(page_string)
    
    #clean regex results to get desired text for each mech: name, weight, chasis variants
    mech_names = [mech_name.group().replace("===", "").strip() for mech_name in mech_results]
    mech_weights = [mech_weight.group().replace("\n", "")[-3:].strip() for mech_weight in tonnage_results]
    chasis_variants = [chasis.group().replace("\n","")[12:].replace(",","").split() for chasis in chasis_results]

    #convert lists to dict as preprocess for converstion to dataframe
    mech_dict = {
        "mechs":mech_names,
        "tonnage":mech_weights,
        "variants":chasis_variants
    }
    mech_df = pd.DataFrame(mech_dict)
    return mech_df

In [95]:
medium_mech_df = get_mech_df(url='https://wiki.mwomercs.com/index.php?title=Assault_Mechs&action=edit')
#Light: https://wiki.mwomercs.com/index.php?title=Light_Mechs&action=edit
#Medium: https://wiki.mwomercs.com/index.php?title=Medium_Mechs&action
#Heavy:https://wiki.mwomercs.com/index.php?title=Heavy_Mechs&action
#Assault: https://wiki.mwomercs.com/index.php?title=Assault_Mechs&action
medium_mech_df

,mechs,tonnage,variants
0,Awesome (AWS),80,"[AWS-8Q, AWS-8R, AWS-8T, AWS-8V, AWS-9M]"
1,Charger (CGR),80,"[CGR-1A1, CGR-1A5, CGR-3K]"
2,Hatamoto (HTM),80,"[HTM-27T, HTM-27W, HTM-28TR]"
3,Victor (VTR),80,"[VTR-9A1, VTR-9B, VTR-9K, VTR-9S]"
4,Zeus (ZEU),80,"[ZEU-5S, ZEU-6S, ZEU-6T, ZEU-9S, ZEU-9S2]"
5,Battlemaster (BLR),85,"[BLR-1D, BLR-1G, BLR-1S, BLR-2C, BLR-3M, BLR-3S]"
6,Stalker (STK),85,"[STK-3F, STK-3FB, STK-3H, STK-4N, STK-5M, STK-5S]"
7,Cyclops (CP),90,"[CP-10-Q, CP-10-Z, CP-11-A, CP-11-A-DC, CP-11-P]"
8,Highlander (HGN),90,"[HGN-732, HGN-732B, HGN-733, HGN-733C, HGN-733P]"
9,Mauler (MAL),90,"[MAL-1P, MAL-1R, MAL-2P, MAL-MX90]"


must pass URL
